In [1]:
import torch
import torch.nn as nn

import yaml
import os

In [2]:
demo_dir = 'demo'
edsr_liif_dir = 'train_edsr-baseline-liif.yaml'
edsr_x2_dir = 'train_edsr-baseline-x2.yaml'

config1 = os.path.join(demo_dir, edsr_liif_dir)
config2 = os.path.join(demo_dir, edsr_x2_dir)

In [3]:
models = {}


def register(name):
    def decorator(cls):
        models[name] = cls
        return cls
    return decorator

@register('liif')
class LIIF(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x


@register('edsr')
def make_edsr():
    return 0


@register('edsr-baseline')
def make_edsr_baseline():
    return 'aaaaa'

print(models)

{'liif': <class '__main__.LIIF'>, 'edsr': <function make_edsr at 0x7f4ab358edc0>, 'edsr-baseline': <function make_edsr_baseline at 0x7f4ab358ef70>}


In [17]:
with open(config1, 'r') as f:
    print(yaml.__version__)
    conf1 = yaml.load(f, Loader=yaml.FullLoader)
    print('config loaded')

5.3.1
config loaded


In [18]:
train_loader = conf1.get('train_dataset')
val_loader = conf1.get('val_dataset')

print("train_loader: {}\n".format(train_loader))
print("val_loader: {}\n".format(val_loader))

train_loader: {'dataset': {'name': 'image-folder', 'args': {'root_path': './load/div2k/DIV2K_train_HR', 'repeat': 20, 'cache': 'in_memory'}}, 'wrapper': {'name': 'sr-implicit-downsampled', 'args': {'inp_size': 48, 'scale_max': 4, 'augment': True, 'sample_q': 2304}}, 'batch_size': 16}

val_loader: {'dataset': {'name': 'image-folder', 'args': {'root_path': './load/div2k/DIV2K_valid_HR', 'first_k': 10, 'repeat': 160, 'cache': 'in_memory'}}, 'wrapper': {'name': 'sr-implicit-downsampled', 'args': {'inp_size': 48, 'scale_max': 4, 'sample_q': 2304}}, 'batch_size': 16}



In [19]:
def make(model_spec, args=None, load_sd=False):
    if args is not None:
        model_args = copy.deepcopy(model_spec['args'])
        model_args.update(args)
    else:
        model_args = model_spec['args']
    model = models[model_spec['name']](**model_args)
    if load_sd:
        model.load_state_dict(model_spec['sd'])
    return model

In [22]:
conf1_model = conf1['model']
print("model: {}\n".format(conf1_model))

# loaded_model = make(conf1_model)

# model_args = copy.deepcopy(conf1_model['args'])
# model_args.update(args)

model_args = conf1_model['args']
print("model_args: {}".format(model_args))
# looks into dictionary with argument
model = models[conf1_model['name']]

model: {'name': 'liif', 'args': {'encoder_spec': {'name': 'edsr-baseline', 'args': {'no_upsampling': True}}, 'imnet_spec': {'name': 'mlp', 'args': {'out_dim': 3, 'hidden_list': [256, 256, 256, 256]}}}}

model_args: {'encoder_spec': {'name': 'edsr-baseline', 'args': {'no_upsampling': True}}, 'imnet_spec': {'name': 'mlp', 'args': {'out_dim': 3, 'hidden_list': [256, 256, 256, 256]}}}
